# Carina Nebula (NGC 3324): Webb vs Hubble

Compare Webb and Hubble images side-by-side, and also sonified "side-by-side" (i.e., panned to the left and right).

Multiple sounds are generated from each image:

- Musical notes triggered when hovering over stars. The note triggered depends on the pixel hue, from red (low note) to blue (high note)
- Background noise level modulated by the local "noise" in the image.
- Background drone modulated by the RGB image bands.

In [ ]:
import numpy as np
import ipycanvas
import ipyevents
import ipytone
import ipywidgets
import skimage

from skimage import filters
from skimage.morphology import disk, binary_dilation

## Load and pre-process images

The image files are not included in this repository. You can download it here:

- Hubble: https://hubblesite.org/contents/media/images/2008/34/2405-Image.html
- Webb: https://webbtelescope.org/contents/media/images/2022/031/01G77PKB8NKR7S8Z6HBXMYATGJ

The Hubble image has been roughly aligned with the Webb image by calibrating `skimage.transform.SimilarityTransform()` on a set of ~100 control points defined by hand.

In [ ]:
hubble, webb = skimage.io.ImageCollection(["data/hubble_export.tif", "data/webb_export.tif"])

In [ ]:
#width = 560
#height = 300
width = 700
height = 400

hubble_r = skimage.transform.resize(hubble, (height, width), preserve_range=True)
webb_r = skimage.transform.resize(webb, (height, width), preserve_range=True)

Create binary masks that extract stars

In [ ]:
def detect_stars(img, threshold=0.025):
    """Simple image filter (+ dilation) to extract stars as a
    binary mask)
    
    """
    bw = skimage.color.rgb2gray(img / 256)
    fltr = filters.rank.median(skimage.util.img_as_ubyte(bw), disk(1))
    mask = bw - fltr / 256 > threshold
    dilated_mask = binary_dilation(mask, disk(1))
    return dilated_mask


hubble_star_mask = detect_stars(hubble_r)
webb_star_mask = detect_stars(webb_r)

Create frequency arrays that determine the musical note to play when hovering on a star

In [ ]:
def hue2freq(hue, freq_range=(100, 4000)):
    """convert hue given on interval [0, 1] to a frequency
    
    The returned frequency is within in the given range.
    The frequency range has periodic boundaries.

    """
    fmin = freq_range[0]
    fmax = freq_range[1]
    ampl = fmax - fmin
    return fmin + ampl * np.sin(hue * np.pi)
    

In [ ]:
hubble_hsv = skimage.color.rgb2hsv(hubble_r / 256)
hubble_freq = hue2freq(hubble_hsv[..., 0])

webb_hsv = skimage.color.rgb2hsv(webb_r / 256)
webb_freq = hue2freq(webb_hsv[..., 0])

Create arrays that determine the level of noise (filter images)

In [ ]:
def detect_noise(img):
    """Simple filter to detect the local amount of "noise"
    in the image.
    
    """
    bw = skimage.color.rgb2gray(img) / 256
    bw_sq = bw * bw

    region = disk(3)
    mean_bw = filters.rank.mean(
        skimage.util.img_as_ubyte(bw), footprint=region
    ).astype(np.float32)
    mean_bw_sq = filters.rank.mean(
        skimage.util.img_as_ubyte(bw_sq), footprint=region
    ).astype(np.float32)

    sq_mean_bw = mean_bw * mean_bw
    std = np.sqrt(sq_mean_bw - mean_bw_sq)
    noise = std / (std.max() * 2)
    
    return noise


hubble_noise = detect_noise(hubble_r)
webb_noise = detect_noise(webb_r)

## Setup canvas

In [ ]:
title_height = 60


def create_mcanvas(img, title):
    mcanvas = ipycanvas.MultiCanvas(3, width=width, height=height + title_height)
    
    background = mcanvas[0]
    background.fill_style = "black"
    background.fill_rect(0, 0, width, height + title_height)
    background.put_image_data(img, x=0, y=title_height)
    
    background.stroke_style = "white"
    background.fill_style = "white"
    background.font = f"{title_height - 40}px sans-serif"
    background.text_baseline = "top"
    background.fill_text(title, 20, 20)
    
    return mcanvas
    

hubble_canvas = create_mcanvas(hubble_r, "HUBBLE [L]")
webb_canvas = create_mcanvas(webb_r, "WEBB [R]")

In [ ]:
def draw_circle(canvas, fill_color):
    canvas.stroke_style = "white"
    canvas.fill_style = fill_color
    canvas.line_width = 2.0
    
    xc = width - 40
    yc = title_height // 2
    canvas.fill_circle(xc, yc, 10)
    canvas.stroke_circle(xc, yc, 10)
    

draw_circle(hubble_canvas[2], "black")
draw_circle(webb_canvas[2], "black")

## Setup sounds

In [ ]:
def multiband_img_sound_setup(pan=0, detune=0):
    reverb = ipytone.Reverb().to_destination()
    panner = ipytone.Panner(pan=pan).connect(reverb)
    
    reverb.wet.value = 0.6
    
    noise_gain = ipytone.Gain(gain=0)
    noise_filter = ipytone.Filter(type="lowpass", frequency=1500)
    noise = ipytone.Noise(type="brown").chain(noise_gain, noise_filter, panner).start()
    
    gain_r = ipytone.Gain(gain=0)
    osc_r = ipytone.FatOscillator(frequency="A2").chain(gain_r, panner).start()
    osc_r.type = "sine"
    osc_r.detune.value = detune
    gain_g = ipytone.Gain(gain=0)
    osc_g = ipytone.FatOscillator(frequency="A4").chain(gain_g, panner).start()
    osc_g.type = "triangle"
    osc_g.detune.value = detune
    gain_b = ipytone.Gain(gain=0)
    osc_b = ipytone.FatOscillator(frequency="C6").chain(gain_b, panner).start()
    osc_b.type = "sine"
    osc_b.detune.value = detune
        
    psynth = ipytone.PolySynth(volume=-4, max_polyphony=10).connect(panner)
    psynth.voice.oscillator.type = "amsine"
    psynth.voice.envelope.release = 1.8
    psynth.voice.envelope.attack = 0.25
    
    return {
        "noise_gain": noise_gain,
        "noise_filter": noise_filter,
        "noise": noise,
        "gain_r": gain_r,
        "osc_r": osc_r,
        "gain_g": gain_g,
        "osc_g": osc_g,
        "gain_b": gain_b,
        "osc_b": osc_b,
        "psynth": psynth,
        "panner": panner,
        "reverb": reverb,
    }


def dispose(sound_setup):
    for widget in sound_setup.values():
        widget.dispose()

In [ ]:
hubble_sound = multiband_img_sound_setup(pan=-1)
webb_sound = multiband_img_sound_setup(pan=1, detune=40)

## Setup mouse event handlers

In [ ]:
hubble_canvas[1].stroke_style = "white"
hubble_canvas[1].line_width = 2.0
webb_canvas[1].stroke_style = "white"
webb_canvas[1].line_width = 2.0


def draw_pointer(x, y):
    for mcv in [hubble_canvas, webb_canvas]:
        mcv[1].clear()
        mcv[1].stroke_circle(x, y + title_height, 15)


def update_circle_color(x ,y):
    t = [
        (hubble_r, hubble_star_mask, hubble_canvas),
        (webb_r, webb_star_mask, webb_canvas),
    ]
    
    for img, mask, mcv in t:
        if mask[y, x]:
            r, g, b = img[y, x].astype(np.uint8)
            color = f"rgb({r}, {g}, {b})"
        else:
            color = "black"
    
        draw_circle(mcv[2], color)


def trigger_note(x, y, mask, freq, hsv, sound):
    if mask[y, x]:
        note = freq[y, x]
        vel = hsv[y, x, 2]
        sound["psynth"].trigger_attack_release(note, 0.1, velocity=vel)


def adjust_gains(x, y, img, sound):
    r, g, b = img[y, x] / 256
    sound["gain_r"].gain.ramp_to(r, 0.2)
    sound["gain_g"].gain.ramp_to(g / 3, 0.2)
    sound["gain_b"].gain.ramp_to(b / 3, 0.2)


def update_sound(x, y):
    trigger_note(x, y, hubble_star_mask, hubble_freq, hubble_hsv, hubble_sound)
    trigger_note(x, y, webb_star_mask, webb_freq, webb_hsv, webb_sound)
    
    hubble_sound["noise_gain"].gain.ramp_to(hubble_noise[y, x], 0.1)
    webb_sound["noise_gain"].gain.ramp_to(webb_noise[y, x], 0.1)
    
    adjust_gains(x, y, hubble_r, hubble_sound)
    adjust_gains(x, y, webb_r, webb_sound)


def action(event):
    x = int(event["relativeX"])
    y = int(event["relativeY"] - title_height)
    
    if y < 0:
        return
    
    draw_pointer(x, y)
    update_circle_color(x, y)
    update_sound(x, y)
    

event = ipyevents.Event(source=hubble_canvas, watched_events=["mousemove"], wait=120)
event.on_dom_event(action)

## Let's play

In [ ]:
ipywidgets.HBox([hubble_canvas, webb_canvas])

## Clean-up

In [ ]:
dispose(hubble_sound)
dispose(webb_sound)

In [ ]:
ipytone.destination.volume.value = 4